In this week's challenge, I will attempt to tame this dataset, by drawing correlation between timestamps, whether the content was shared or not, and the view count. We can start by importing the data first.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
dataset = pd.read_csv('BX-Users.csv', encoding='ISO-8859–1', on_bad_lines='skip')

In [3]:
!wc -l BX-Book-Ratings.csv

 1149781 BX-Book-Ratings.csv


In [4]:
!tail -1149781 BX-Book-Ratings.csv > BX-Book-Ratings-Clean.csv

In [5]:
#Some idiosyncracies with the data require me to skip bad lines and seperate columns manually
#Had to encode the CSV to work
#Update: pandas has built in arguments to handle this when reading in data.

ratings = pd.read_csv('BX-Book-Ratings-Clean.csv', encoding='ISO-8859–1',on_bad_lines='skip',quotechar='"',sep=";",escapechar= "\\")
books = pd.read_csv('BX_Books.csv', encoding='ISO-8859–1',on_bad_lines='skip',quotechar='"',sep=";",escapechar= "\\")
users = pd.read_csv('BX-Users.csv', encoding='ISO-8859–1',on_bad_lines='skip',quotechar='"',sep=";",escapechar= "\\")


In [6]:
#display 5 rows of Books
books.head()


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
books.shape

(271379, 8)

In [9]:
#display the ratings
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [10]:
ratings.shape

(1149780, 3)

In [11]:
#display the users
users.head()



,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [12]:
users.shape

(278858, 3)

Starting with the Book Data,lets drop columns which we know aren't going to be of much use. After this we can start evaluating comparative data :)


In [30]:
#code to drop cols and return an updated 'books' dataframe.
books.drop(['Image-URL-S','Image-URL-M', 'Image-URL-L'], axis=1,inplace=True)

books.head

KeyError: "['Image-URL-S', 'Image-URL-M', 'Image-URL-L'] not found in axis"

In [22]:
#code to check if nulls exist in Data Frame
print(books['Book-Title'].isnull().values.any())
print(books['Book-Author'].isnull().values.any())
print(books['Year-Of-Publication'].isnull().values.any())
print(books['Publisher'].isnull().values.any())

print()

#code to count the # of Nulls in respective columns.
print(books['Book-Author'].isnull().sum())
print(books['Publisher'].isnull().sum())


False
True
False
True

1
2


In [27]:
#We are going to fix occurences of nulls and set them to 'other' instead of Null.


print(books.loc[(books['Book-Author'].isnull()),:])
print(books.loc[(books['Publisher'].isnull()),:])
books['Book-Author']=books['Book-Author'].fillna("other")
books['Publisher']=books['Book-Author'].fillna("none provided")

              ISBN                                         Book-Title  \
187700  9627982032  The Credit Suisse Guide to Managing Your Perso...   

       Book-Author  Year-Of-Publication      Publisher Book-author  
187700         NaN                 1995  none provided       other  
Empty DataFrame
Columns: [ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher, Book-author]
Index: []


In [31]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Book-author            0
dtype: int64

In [32]:
books.head(-1)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Book-author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Mark P. O. Morford,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright,2001,Richard Bruce Wright,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este,1991,Carlo D'Este,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Gina Bari Kolata,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,E. J. W. Barber,E. J. W. Barber
...,...,...,...,...,...,...
271373,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Robin Wright,Robin Wright
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Paula Danziger,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat,1991,Teri Sloat,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,Christine Wicker,Christine Wicker


Next we're going to apply the same investigative methods to the Ratings and the Users. By the end I expect to find a correlation between the datasets


In [33]:
"""
My first idea is to exclude NaN value ages and also any strange age values. 
I am more interested in age groups 10-90 anything above or below this range doesn't seem feasible or worthwhile.
"""

users.shape

(278858, 3)

In [34]:
users["Age"].unique()
#as you can see, there are people who are above the age 100 and below 10 years old.
#code to exclude them below

array([ nan,  18.,  17.,  61.,  26.,  14.,  25.,  19.,  46.,  55.,  32.,
        24.,  20.,  34.,  23.,  51.,  31.,  21.,  44.,  30.,  57.,  43.,
        37.,  41.,  54.,  42.,  50.,  39.,  53.,  47.,  36.,  28.,  35.,
        13.,  58.,  49.,  38.,  45.,  62.,  63.,  27.,  33.,  29.,  66.,
        40.,  15.,  60.,   0.,  79.,  22.,  16.,  65.,  59.,  48.,  72.,
        56.,  67.,   1.,  80.,  52.,  69.,  71.,  73.,  78.,   9.,  64.,
       103., 104.,  12.,  74.,  75., 231.,   3.,  76.,  83.,  68., 119.,
        11.,  77.,   2.,  70.,  93.,   8.,   7.,   4.,  81., 114., 230.,
       239.,  10.,   5., 148., 151.,   6., 101., 201.,  96.,  84.,  82.,
        90., 123., 244., 133.,  91., 128.,  94.,  85., 141., 110.,  97.,
       219.,  86., 124.,  92., 175., 172., 209., 212., 237.,  87., 162.,
       100., 156., 136.,  95.,  89., 106.,  99., 108., 210.,  88., 199.,
       147., 168., 132., 159., 186., 152., 102., 116., 200., 115., 226.,
       137., 207., 229., 138., 109., 105., 228., 18

In [35]:
#turn unwanted values into NaN
users.loc[ (users['Age'] > 90) | (users['Age']<10) ] = np.nan
#drop NaN values.
users.dropna(inplace=True)
users.shape

(166597, 3)

In [36]:
users.head(-1)

,User-ID,Location,Age
1,2.0,"stockton, california, usa",18.0
3,4.0,"porto, v.n.gaia, portugal",17.0
5,6.0,"santa monica, california, usa",61.0
9,10.0,"albacete, wisconsin, spain",26.0
10,11.0,"melbourne, victoria, australia",14.0
...,...,...,...
278845,278846.0,"toronto, ontario, canada",23.0
278848,278849.0,"georgetown, ontario, canada",23.0
278850,278851.0,"dallas, texas, usa",33.0
278851,278852.0,"brisbane, queensland, australia",32.0


that does it for User Data, lets skim through ratings.

In [37]:
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [40]:
"""
Looking back at the Ratings Data, I see that ISBN is used both in the Books set as well as here. 
It appears USER-ID is also connected which is equally interesting and important.

To tie all of this research together, I will evaluate Ratings according to the other sets I have sifted through.

First I want to set the amount of rows according to ISBNs in the Books set.
"""

sorted_ratings = ratings[ratings.ISBN.isin(books.ISBN)]
sorted_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149774,276704,0876044011,0
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10


In [49]:
#lets do the same for user ID
sorted_ratings = sorted_ratings[sorted_ratings['User-ID'].isin(users['User-ID'])]
sorted_ratings

,User-ID,ISBN,Book-Rating
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
16,276747,0060517794,9
...,...,...,...
1149756,276690,0590581066,0
1149757,276690,0590907301,0
1149776,276706,0679447156,0
1149777,276709,0515107662,10


In [ ]:
print(psorted_ratings.shape)
print(ratings)